**Computational Health Laboratory Project, A.Y. 2021/2022**

**Authors:** Niko Dalla Noce, Alessandro Ristori, Andrea Zuppolini

**Project:** Starting fron one or more genes, extract from interaction databases the genes they interact with. Using the expanded gene set, perform pathway analysis and obtain all disease pathways in which the genes appear. Merge the pathways to obtain a larger graph. Perform further network analysis to extract central biomarkers and communities beyond pathways. Compute a distance between the initial gene set and the various pathways (diseases).

# **CHL Project, Network Analysis**

## **Colab setup**
Takes care of the project setup on Colab.

In [1]:
if 'google.colab' in str(get_ipython()):
    import subprocess
    out_clone = subprocess.run(["git", "clone", "https://github.com/nikodallanoce/ComputationalHealthLaboratory"], text=True, capture_output=True)
    print("{0}{1}".format(out_clone.stdout, out_clone.stderr))
    %cd ComputationalHealthLaboratory

Cloning into 'ComputationalHealthLaboratory'...

/content/ComputationalHealthLaboratory


## **Protein-Protein network**
Build the protein-to-protein network and link each node to its diseases.


In [2]:
import numpy as np
import os
import pandas as pd
from tqdm.notebook import tqdm
import networkx as nx

We assume that you have already done pathway enrichment on notebook **0_Pathway_Enrichment** and, therefore, all the datasets needed here are available. If so, then load everything.

In [35]:
nodes = pd.read_csv("datasets/genes.csv", sep=",", index_col=0)["0"]
df_diseases = pd.read_csv("datasets/diseases_pathways.csv", sep=",", index_col=0)
interactions = pd.read_csv("datasets/interactions.csv", sep=",", index_col=0)
diseases = dict()
for i, disease in df_diseases.iterrows():
    disease_genes = disease['Genes'].split(";")
    term = disease['Term']
    diseases[i] = {"name": term, "genes": disease_genes}

Build the graph and fill it with its nodes (the proteins coming from the dataset).

In [30]:
# Build the graph
protein_graph = nx.Graph(name='Protein Interactions Graph')

# Build the edges
for node in nodes:
    protein_graph.add_node(node, diseases=[])  # Each node will have a list with the disease pathways it belongs to

Insert into the node their respective diseases.

In [31]:
for i, disease in diseases.items():
    disease_genes = disease['genes']
    for gene in disease_genes:
        protein_graph.nodes[gene]["diseases"].append(i)

There could be nodes without any diseases, they still need to be kept into the network.

In [32]:
nodes_no_disease = list()
for node in protein_graph.nodes:
    if len(protein_graph.nodes[node]["diseases"])==0:
        nodes_no_disease.append(str(node))

In [33]:
print("Nodes without diseases: {0}".format(len(nodes_no_disease)))

Nodes without diseases: 5101


Then, build the edges, it's straightforward as the nodes are known.

In [37]:
for _, interaction in interactions.iterrows():
    first_protein, second_protein = interaction[0], interaction[1]  # Proteins involved in the interaction

    # Build the edge
    protein_graph.add_edge(first_protein, second_protein)

At last, save the graph.

In [38]:
nx.write_gpickle(protein_graph,'protein_graph.gpickle')

## **Metrics**
Metrics needed to compare the various diseases and proteins.

Load the graph if already built previously and is not available currently.

In [39]:
if os.path.exists("datasets/protein_graph.gpickle") and not "protein_graph" in locals():
    protein_graph = nx.read_gpickle("datasets/protein_graph.gpickle")
elif not "protein_graph" in locals():
    raise ValueError("It was not possible to find the graph, build it from the previous step")

### **Size of largest pathway component**
Fraction of disease proteins that lie in the disease's largest pathway component (i.e., the relative size of the largest connected component (LCC) of the disease).

In [40]:
def largest_conn_comp(diseases: dict) -> list:
    lcc_score = list()
    for _, disease in tqdm(diseases.items()):
        sub_graph = protein_graph.subgraph(disease['genes'])  # Subgraph of the current disease
        largest_cc = max(nx.connected_components(sub_graph), key=len)
        lcc_score.append(len(largest_cc) / len(sub_graph.nodes()))
    
    return lcc_score

In [41]:
df_diseases.insert(len(df_diseases.columns), "LCC Score", largest_conn_comp(diseases), True)

  0%|          | 0/589 [00:00<?, ?it/s]

In [42]:
df_diseases.tail()

,Term,Overlap,P-value,Adjusted P-value,Genes,LCC Score
584,Chronic otitis media,55/69,0.005896,0.098950,IGHM;CD81;WIPF1;FMR1;DOCK8;CHD7;JMJD1C;COMT;GT...,0.018182
585,Inadequate arch length for tooth size,47/58,0.005953,0.099228,AMER1;SETD5;NOTCH3;TRIO;RPL10;SATB2;GNAI3;PLOD...,0.234043
586,Tooth Crowding,47/58,0.005953,0.099228,AMER1;SETD5;NOTCH3;TRIO;RPL10;SATB2;GNAI3;PLOD...,0.234043
587,Tooth mass arch size discrepancy,47/58,0.005953,0.099228,AMER1;SETD5;NOTCH3;TRIO;RPL10;SATB2;GNAI3;PLOD...,0.234043
588,Tooth size discrepancy,47/58,0.005953,0.099228,AMER1;SETD5;NOTCH3;TRIO;RPL10;SATB2;GNAI3;PLOD...,0.234043


### **Distance of pathway components** For each pair of pathway components, we calculate the average shortest path length between each set of proteins, and then, the average of this is taken over all pairs of the components.

In [43]:
from numpy.ma.core import mean

def distance_pathway_comps(diseases: dict) -> list:
    dpc_score = list()
    for _, disease in tqdm(diseases.items()):
        sub_graph = protein_graph.subgraph(disease['genes'])
        conn_comps = list(nx.connected_components(sub_graph))
        distances = list()
        for i, comp in enumerate(conn_comps):
            for j in range(i+1, len(conn_comps)):
                dist = 0
                for first_comp_protein in comp:
                    for second_comp_protein in conn_comps[j]:
                        dist += nx.shortest_path_length(protein_graph, source=first_comp_protein, target=second_comp_protein)
                
                distances.append(dist / (len(comp) * len(conn_comps[j])))
        dpc_score.append(mean(distances))
    
    return dpc_score

In [48]:
if os.path.exists("datasets/mean_distances.csv") and not "df_mean_distances" in locals():
    df_mean_distances = pd.read_csv("datasets/mean_distances.csv", sep=",", index_col=0)
elif not "df_mean_distances" in locals():
    df_mean_distances = pd.DataFrame(distance_pathway_comps(diseases))
    df_mean_distances.to_csv('datasets/mean_distances.csv')

In [45]:
df_diseases["DPC Score"] = df_mean_distances
df_diseases.tail()

,Term,Overlap,P-value,Adjusted P-value,Genes,LCC Score,DPC Score
584,Chronic otitis media,55/69,0.005896,0.098950,IGHM;CD81;WIPF1;FMR1;DOCK8;CHD7;JMJD1C;COMT;GT...,0.018182,2.678114
585,Inadequate arch length for tooth size,47/58,0.005953,0.099228,AMER1;SETD5;NOTCH3;TRIO;RPL10;SATB2;GNAI3;PLOD...,0.234043,2.693694
586,Tooth Crowding,47/58,0.005953,0.099228,AMER1;SETD5;NOTCH3;TRIO;RPL10;SATB2;GNAI3;PLOD...,0.234043,2.693694
587,Tooth mass arch size discrepancy,47/58,0.005953,0.099228,AMER1;SETD5;NOTCH3;TRIO;RPL10;SATB2;GNAI3;PLOD...,0.234043,2.693694
588,Tooth size discrepancy,47/58,0.005953,0.099228,AMER1;SETD5;NOTCH3;TRIO;RPL10;SATB2;GNAI3;PLOD...,0.234043,2.693694


### **Network modularity**
Fraction of edges that fall within/outside the pathway minus the expected fraction if edges were randomly distributed:
\begin{equation}
Q_d = 1/(2m) \sum_{ij} (I((i, j) ∈ E) − \frac{k_ik_j}{
2m})δ(p_i, p_j)
\end{equation}
where $k_i$ is the degree of $i$, and $δ(p_i, p_j)$ is 1 if $p_i$ and $p_j$ are equal and 0 otherwise.


In [46]:
def intersection(lst1, lst2):
    inters = list()
    if not (len(lst1)==0 or len(lst2)==0):
        set1 = set(lst1)
        inters = [elem for elem in lst2 if elem in set1]
    return inters

In [47]:
def network_modularity(protein_graph: nx.Graph, diseases: dict) -> list:
    m = protein_graph.number_of_edges()
    one_m = 1/(2*m)
    Q = list()
    for _, disease in tqdm(diseases.items()):
        sub_graph = protein_graph.subgraph(disease['genes'])
        disease_nodes = list(sub_graph.nodes())
        Q_dis = 0
        for i, node_i in enumerate(disease_nodes):
            for j in range(i+1, len(disease_nodes)):
                node_j = disease_nodes[j]
                a = protein_graph.number_of_edges(node_i, node_j)
                k_i=protein_graph.degree[node_i]
                k_j=protein_graph.degree[node_j]
                Q_dis += a - (k_i*k_j)/(2*m)
        
        Q.append(one_m * Q_dis)
    
    return Q 

In [49]:
if os.path.exists("datasets/modularities.csv") and not "df_modularities" in locals():
    df_modularities = pd.read_csv("datasets/modularities.csv", sep=",", index_col=0)
elif not "df_modularities" in locals():
    df_modularities = pd.DataFrame(network_modularity(protein_graph, diseases))
    df_modularities.to_csv('datasets/modularities.csv')

In [50]:
df_diseases["Modularity"] = df_modularities
df_diseases.tail()

,Term,Overlap,P-value,Adjusted P-value,Genes,LCC Score,DPC Score,Modularity
584,Chronic otitis media,55/69,0.005896,0.098950,IGHM;CD81;WIPF1;FMR1;DOCK8;CHD7;JMJD1C;COMT;GT...,0.018182,2.678114,-0.000008
585,Inadequate arch length for tooth size,47/58,0.005953,0.099228,AMER1;SETD5;NOTCH3;TRIO;RPL10;SATB2;GNAI3;PLOD...,0.234043,2.693694,0.000004
586,Tooth Crowding,47/58,0.005953,0.099228,AMER1;SETD5;NOTCH3;TRIO;RPL10;SATB2;GNAI3;PLOD...,0.234043,2.693694,0.000004
587,Tooth mass arch size discrepancy,47/58,0.005953,0.099228,AMER1;SETD5;NOTCH3;TRIO;RPL10;SATB2;GNAI3;PLOD...,0.234043,2.693694,0.000004
588,Tooth size discrepancy,47/58,0.005953,0.099228,AMER1;SETD5;NOTCH3;TRIO;RPL10;SATB2;GNAI3;PLOD...,0.234043,2.693694,0.000004


### **Save the updated pathways with their scores.**

In [51]:
df_diseases.to_csv("datasets/diseases_scores.csv")

## **Network biomarkers**

In [ ]:
nodes_degree = pd.DataFrame(protein_graph.degree(list(protein_graph.nodes())), columns=['protein', 'degree'])
nodes_degree = nodes_degree.sort_values(by='degree', ascending=False)

In [ ]:
nodes_degree[nodes_degree['degree']==1].count()

In [ ]:
nodes_degree.iloc[0:100,:]